In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np

In [3]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [4]:
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [5]:
# 파일 경로
file_path = '/content/drive/MyDrive/PC - Desktop/2024-1. 안교수님-지식그래프-팀플/data/기록물.xlsx'
task_path = '/content/drive/MyDrive/PC - Desktop/2024-1. 안교수님-지식그래프-팀플/data/제주특별자치도개발공사 기록관리기준표(2023.12. 기준)_고시.xlsx'

In [6]:
# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

In [7]:
task = pd.read_excel(task_path, header=1)

In [8]:
df.head(2)

,등록구분,문서번호,제목,기안(접수)자,결재(담당)자,기안일자,등록일자,수신(발신)자,시행일자,공개여부,단위업무명,보존기간
0,일반문서 생산/보고,시설환경팀-6301,차량배차신청서,이지연,이호준,2023.12.29 15:25,2023-12-31 22:10:00,내부결재,NaN,공개,삼다수사업장관용차량관리,10년
1,일반문서 생산/보고,시설환경팀-6300,차량운행일지,이지연,이호준,2023.12.29 16:14,2023-12-31 22:10:00,내부결재,NaN,공개,삼다수사업장관용차량관리,10년


In [9]:
print(df.shape)

(113677, 12)


In [10]:
df_100 = df.iloc[:100, :]
print(df_100.shape)

(100, 12)


In [11]:
df_100.head(2)

,등록구분,문서번호,제목,기안(접수)자,결재(담당)자,기안일자,등록일자,수신(발신)자,시행일자,공개여부,단위업무명,보존기간
0,일반문서 생산/보고,시설환경팀-6301,차량배차신청서,이지연,이호준,2023.12.29 15:25,2023-12-31 22:10:00,내부결재,NaN,공개,삼다수사업장관용차량관리,10년
1,일반문서 생산/보고,시설환경팀-6300,차량운행일지,이지연,이호준,2023.12.29 16:14,2023-12-31 22:10:00,내부결재,NaN,공개,삼다수사업장관용차량관리,10년


In [12]:
task.head(2)

,부서명,임시단위업무코드,대기능코드,대기능명,중기능코드,중기능명,소기능코드,단위과제명,단위과제설명,보존기간,보존기간책정사유
0,부서공통,0,0,0,0.0,0.0,0.0,부서감사수감및조치,"감사수감과 감사결과 접수 및 조치, 청렴에 관한 업무",5,감사수감 및 결과조치에 관한 기록물은 국가기록원 단위과제별 공통업무 보존기간표(감사...
1,부서공통,0,0,0,0.0,0.0,0.0,부서국회및도의회대응,국회 및 도의회 요구자료 제출 등 국회 및 도의회 요청사항 관리 및 대응,3,국회 대응에 관한 기록물은 국가기록원 단위과제별 공통업무 보존기간표(국회 관련 업무...


In [13]:
print(task.shape)

(848, 11)


# hugging face을 활용한 단어 임베딩


In [14]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [15]:
# !pip install konlpy

In [16]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 문장을 임베딩으로 변환
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
embedding1 = model.encode(df_100['제목'].to_list())
embedding2 = model.encode(task['단위과제설명'].to_list())

In [18]:
print(embedding1)
print('=========')
print(embedding2)

[[ 0.2118285   0.0858767  -0.04985491 ...  0.06521714 -0.16035526
   0.12123047]
 [ 0.25727165  0.1650084  -0.01154575 ...  0.01050949  0.06429823
  -0.02960369]
 [-0.12637337  0.2193074   0.22178473 ...  0.04796952 -0.08699343
   0.1264091 ]
 ...
 [ 0.16375598  0.3262267   0.02594275 ... -0.21014152  0.18533717
  -0.04106846]
 [ 0.19516309  0.08024287  0.04418215 ... -0.10153656  0.04782
   0.0692996 ]
 [-0.01781897  0.11884192 -0.10685503 ... -0.08931167 -0.05446442
   0.12959431]]
[[ 0.0255482   0.23560931 -0.03344118 ...  0.00205874  0.04193786
   0.07779605]
 [ 0.07250697  0.0439762   0.13362974 ...  0.20257868 -0.21840642
   0.10476603]
 [ 0.15475507  0.13403495 -0.00203249 ... -0.10120111  0.09127855
   0.11497365]
 ...
 [ 0.12380166  0.2672523   0.06333428 ... -0.03005621  0.17629468
   0.08207184]
 [-0.0575951   0.01278444  0.06971765 ...  0.0708066   0.13031936
   0.14535648]
 [ 0.1232355   0.00967082 -0.08738472 ... -0.01894964  0.03583636
   0.244947  ]]


In [19]:
# 임베딩 간의 유사도 계산
similarity = cosine_similarity(embedding1, embedding2)
print(len(similarity))
print("유사도 행렬 shape:", similarity.shape)

100
유사도 행렬 shape: (100, 848)


In [20]:
rematching_task = []
# embedding1의 모든 행 반복
for i, first_embedding in enumerate(embedding1):
    # 최대 유사도와 해당 인덱스 저장 변수
    max_similarity = -1
    max_index = -1

    # embedding1의 현재 행과 embedding2 모든 행 간 유사도 비교
    for j, second_embedding in enumerate(embedding2):
        # 코사인 유사도 계산
        similarity = np.dot(first_embedding, second_embedding) / (np.linalg.norm(first_embedding) * np.linalg.norm(second_embedding))

        # 최대 유사도 갱신
        if similarity > max_similarity:
            max_similarity = similarity
            max_index = j
    rematching_task.append(max_index)
    # 결과 출력
    print(f"[첫번째 행 {i+1}] 최대 유사도: {max_similarity:.4f}, 해당 인덱스: {max_index}")


[첫번째 행 1] 최대 유사도: 0.7422, 해당 인덱스: 573
[첫번째 행 2] 최대 유사도: 0.8413, 해당 인덱스: 604
[첫번째 행 3] 최대 유사도: 0.6409, 해당 인덱스: 10
[첫번째 행 4] 최대 유사도: 0.9243, 해당 인덱스: 729
[첫번째 행 5] 최대 유사도: 0.8955, 해당 인덱스: 563
[첫번째 행 6] 최대 유사도: 0.9165, 해당 인덱스: 15
[첫번째 행 7] 최대 유사도: 0.7422, 해당 인덱스: 573
[첫번째 행 8] 최대 유사도: 0.8413, 해당 인덱스: 604
[첫번째 행 9] 최대 유사도: 0.6512, 해당 인덱스: 254
[첫번째 행 10] 최대 유사도: 0.8129, 해당 인덱스: 594
[첫번째 행 11] 최대 유사도: 0.6794, 해당 인덱스: 406
[첫번째 행 12] 최대 유사도: 0.7982, 해당 인덱스: 307
[첫번째 행 13] 최대 유사도: 0.6022, 해당 인덱스: 774
[첫번째 행 14] 최대 유사도: 0.8202, 해당 인덱스: 526
[첫번째 행 15] 최대 유사도: 0.7322, 해당 인덱스: 48
[첫번째 행 16] 최대 유사도: 0.9454, 해당 인덱스: 604
[첫번째 행 17] 최대 유사도: 0.7032, 해당 인덱스: 569
[첫번째 행 18] 최대 유사도: 0.7918, 해당 인덱스: 197
[첫번째 행 19] 최대 유사도: 0.9701, 해당 인덱스: 604
[첫번째 행 20] 최대 유사도: 0.6417, 해당 인덱스: 537
[첫번째 행 21] 최대 유사도: 0.7701, 해당 인덱스: 428
[첫번째 행 22] 최대 유사도: 0.8156, 해당 인덱스: 190
[첫번째 행 23] 최대 유사도: 0.9149, 해당 인덱스: 810
[첫번째 행 24] 최대 유사도: 0.9759, 해당 인덱스: 362
[첫번째 행 25] 최대 유사도: 0.7211, 해당 인덱스: 64
[첫번째 행 26] 최대 유사도: 0.8003, 해당 인덱스: 13


In [21]:
# '재매칭_단위업무명' 열을 만들고 초기값으로 빈 문자열 할당
df_100['재매칭_단위업무명'] = ""

# '재매칭_단위업무명' 열에 값을 할당
for i, n in enumerate(rematching_task):
    df_100.at[i, '재매칭_단위업무명'] = task['단위과제명'].loc[n]

<ipython-input-21-552c3e12b9fc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['재매칭_단위업무명'] = ""


In [22]:
print(df_100['단위업무명'], df_100['재매칭_단위업무명'])

0     삼다수사업장관용차량관리 
1     삼다수사업장관용차량관리 
2         부서교육및연수관리
3        삼다수사업장환경관리
4        삼다수사업장환경관리
          ...      
95             자금관리
96    임시사무연구동안전보건관리
97    임시사무연구동안전보건관리
98           부서서무업무
99        부서인사및복무관리
Name: 단위업무명, Length: 100, dtype: object 0            삼다수사업장 차량관리
1             삼다수사업장예초관리
2              부서교육및연수관리
3              잠재오염원조사관리
4     삼다수사업장유틸리티급배기펜유지보수
             ...        
95               주거복지협의회
96             임시사무동보건업무
97            삼다수사업장예초관리
98           삼다수사업장TPM관리
99              인력운영계획수립
Name: 재매칭_단위업무명, Length: 100, dtype: object


In [23]:
# 정확하게 일치하는 횟수 세기
correct_count = sum(df_100['단위업무명'] == df_100['재매칭_단위업무명'])

# 전체 예측의 수
total_predictions = len(df_100)

# 정확도 계산
accuracy = correct_count / total_predictions

print("정확도:", accuracy)
print(f'맞춘 개수 / 전체 개수 : {correct_count}/{total_predictions}')

정확도: 0.02
맞춘 개수 / 전체 개수 : 2/100


## 기록물 전체 단위업무명 재매칭

In [24]:
import time

# 코드 실행 시작 시간 기록
start_time = time.time()

# 임베딩 생성
embedding1 = model.encode(df['제목'].to_list())
embedding2 = model.encode(task['단위과제설명'].to_list())

# 코드 실행 종료 시간 기록
end_time = time.time()

# 코드 실행 시간 계산
execution_time = end_time - start_time
print("코드 실행 시간:", execution_time, "초")

코드 실행 시간: 3544.6687643527985 초


In [25]:
# 임베딩 간의 유사도 계산
start_time = time.time()
similarity = cosine_similarity(embedding1, embedding2)
end_time = time.time()
execution_time = end_time - start_time
print("코드 실행 시간:", execution_time, "초")
print(len(similarity))
print("유사도 행렬 shape:", similarity.shape)

코드 실행 시간: 1.6970248222351074 초
113677
유사도 행렬 shape: (113677, 848)


In [26]:
rematching_task = []
# embedding1의 모든 행 반복
for i, first_embedding in enumerate(embedding1):
    # 최대 유사도와 해당 인덱스 저장 변수
    max_similarity = -1
    max_index = -1

    # embedding1의 현재 행과 embedding2 모든 행 간 유사도 비교
    for j, second_embedding in enumerate(embedding2):
        # 코사인 유사도 계산
        similarity = np.dot(first_embedding, second_embedding) / (np.linalg.norm(first_embedding) * np.linalg.norm(second_embedding))

        # 최대 유사도 갱신
        if similarity > max_similarity:
            max_similarity = similarity
            max_index = j
    rematching_task.append(max_index)
    # 결과 출력
    print(f"[첫번째 행 {i+1}] 최대 유사도: {max_similarity:.4f}, 해당 인덱스: {max_index}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[첫번째 행 108678] 최대 유사도: 0.8719, 해당 인덱스: 564
[첫번째 행 108679] 최대 유사도: 0.8809, 해당 인덱스: 810
[첫번째 행 108680] 최대 유사도: 0.7140, 해당 인덱스: 19
[첫번째 행 108681] 최대 유사도: 0.8668, 해당 인덱스: 627
[첫번째 행 108682] 최대 유사도: 0.6483, 해당 인덱스: 13
[첫번째 행 108683] 최대 유사도: 0.9871, 해당 인덱스: 379
[첫번째 행 108684] 최대 유사도: 0.8382, 해당 인덱스: 155
[첫번째 행 108685] 최대 유사도: 0.7220, 해당 인덱스: 122
[첫번째 행 108686] 최대 유사도: 0.8118, 해당 인덱스: 380
[첫번째 행 108687] 최대 유사도: 0.8118, 해당 인덱스: 380
[첫번째 행 108688] 최대 유사도: 0.5082, 해당 인덱스: 822
[첫번째 행 108689] 최대 유사도: 0.8063, 해당 인덱스: 108
[첫번째 행 108690] 최대 유사도: 0.6642, 해당 인덱스: 468
[첫번째 행 108691] 최대 유사도: 0.7733, 해당 인덱스: 327
[첫번째 행 108692] 최대 유사도: 0.8878, 해당 인덱스: 845
[첫번째 행 108693] 최대 유사도: 0.8410, 해당 인덱스: 162
[첫번째 행 108694] 최대 유사도: 0.7560, 해당 인덱스: 108
[첫번째 행 108695] 최대 유사도: 0.7203, 해당 인덱스: 175
[첫번째 행 108696] 최대 유사도: 0.5689, 해당 인덱스: 553
[첫번째 행 108697] 최대 유사도: 0.7574, 해당 인덱스: 316
[첫번째 행 108698] 최대 유사도: 0.7574, 해당 인덱스: 316
[첫번째 행 108699] 최대 유사도: 0.6145, 해당 인덱스: 438
[첫번째 행 108700] 최대 유사

In [ ]:
# '재매칭_단위업무명' 열을 만들고 초기값으로 빈 문자열 할당
df['재매칭_단위업무명'] = ""

# '재매칭_단위업무명' 열에 값을 할당
for i, n in enumerate(rematching_task):
    df.at[i, '재매칭_단위업무명'] = task['단위과제명'].loc[n]


In [ ]:
print(df['단위업무명'], df['재매칭_단위업무명'])

In [29]:
# 정확하게 일치하는 횟수 세기
correct_count = sum(df['단위업무명'] == df['재매칭_단위업무명'])

# 전체 예측의 수
total_predictions = len(df)

# 정확도 계산
accuracy = correct_count / total_predictions

print("정확도:", accuracy)
print(f'맞춘 개수 / 전체 개수 : {correct_count}/{total_predictions}')

정확도: 0.037632942459776386
맞춘 개수 / 전체 개수 : 4278/113677


In [30]:
df.to_excel('rematching_task.xlsx', index=False)